In [13]:
#trees.py
import pandas as pd
from math import log
def calShannonEnt(dataSet):
    numEntries = len(dataSet)
    labelCounts = {}
    #为所有可能的分类创建字典
    for featVec in dataSet:
        currentLabel = featVec[-1]
        if currentLabel not in labelCounts.keys():
            labelCounts[currentLabel] = 0
        labelCounts[currentLabel] += 1
    print('labelCounts',labelCounts)
    shannonEnt = 0.0
    for key in labelCounts:
        #计算熵，先求p
        prob = float(labelCounts[key])/numEntries
        shannonEnt -= prob *log(prob,2)
    return shannonEnt


In [14]:
def createDataSet():
	dataSet = [
			[0, 0, 0, 0, 'no'],										
			[0, 0, 0, 1, 'no'],
			[0, 1, 0, 1, 'yes'],
			[0, 1, 1, 0, 'yes'],
			[0, 0, 0, 0, 'no'],
			[1, 0, 0, 0, 'no'],
			[1, 0, 0, 1, 'no'],
			[1, 1, 1, 1, 'yes'],
			[1, 0, 1, 2, 'yes'],
			[1, 0, 1, 2, 'yes'],
			[2, 0, 1, 2, 'yes'],
			[2, 0, 1, 1, 'yes'],
			[2, 1, 0, 1, 'yes'],
			[2, 1, 0, 2, 'yes'],
			[2, 0, 0, 0, 'no']
			]
	labels = ['F1-AGE', 'F2-WORK', 'F3-HOME', 'F4-LOAN']		
	return dataSet, labels


In [15]:
myData,labels = createDataset()
print("数据集：{}\n 标签：{}".format(myData,labels))
print("该数据集下的香农熵为:{}".format(calShannonEnt(myData)))



数据集：[[0, 0, 0, 0, 'no'], [0, 0, 0, 1, 'yes'], [0, 1, 0, 1, 'yes'], [0, 1, 1, 0, 'no'], [0, 0, 0, 0, 'no'], [1, 0, 0, 0, 'no'], [1, 0, 0, 1, 'no'], [1, 1, 1, 1, 'yes'], [1, 0, 1, 2, 'yes'], [1, 0, 1, 2, 'yes'], [2, 0, 1, 2, 'yes'], [2, 0, 1, 1, 'yes'], [2, 1, 0, 1, 'yes'], [2, 1, 0, 2, 'yes'], [2, 0, 0, 0, 'no']]
 标签：['F1-AGE', 'F2-WORK', 'F3-HOME', 'F4-LOAN']
labelCounts {'no': 6, 'yes': 9}
该数据集下的香农熵为:0.9709505944546686


In [16]:
# myData[0][-1] = 'yes'
# print('数据为:',myData)
# print("该数据集下的香农熵为:",calShannonEnt(myData))


In [17]:
# 根据属性及其属性值划分数据集
def splitDataSet(dataSet, axis, value):
    '''dataSet : 待划分的数据集
        axis : 属性及特征
        value : 属性值及特征的hasattr值'''
    retDataSet = []
    for featVet in dataSet:
        if featVet[axis] == value:
            reducedFeatVec = featVet[:axis]
            reducedFeatVec.extend(featVet[axis+1:])
            retDataSet.append(reducedFeatVec)
    return retDataSet


In [18]:
# print('划分前的数据集：',myData)
# print('按照“离开水是否能生存”为划分属性')
# print('划分前的数据集：',splitDataSet(myData,0,0))
# print('划分前的数据集：',splitDataSet(myData,0,1))

In [19]:
# 选择最好的数据集划分方式，及根绝信息增益选择划分属性
def chooseBestFeatureToSplit(dataSet):
    numFeatures = len(dataSet[0]) - 1
    baseEntropy = calShannonEnt(dataSet)
    bestInfoGain, bestFeature = 0, -1
    for i in range(numFeatures):
        featList = [example[i] for example in dataSet]
        uniqueVals = set(featList)
        newEntropy = 0.0
        # 计算每种划分方式的信息熵
        for value in uniqueVals:
            subDataSet = splitDataSet(dataSet, i, value)
            prob = len(subDataSet) / float(len(dataSet))
            newEntropy += prob * calShannonEnt(subDataSet)
        infoGain = baseEntropy - newEntropy
        if(infoGain > bestInfoGain):
            bestInfoGain = infoGain
            bestFeature = i
    return bestFeature


In [20]:
# 找到出现次数最多的分类名称
import operator

def majorityCnt(classList):
    classCount = {}
    for vote in classList:
        if vote not in classCount.keys():
            classCount[vote] = 0
        classCount[vote] += 1
        sortedClassCount = sorted(
            classCount.items(), key=operator.itemgetter(1), reverse=True)
        return sortedClassCount[0][0]


In [21]:
# 创建树的函数
def creatTree(dataSet, labels):
    classList = [example[-1] for example in dataSet]
    print(classList)
    # 类别完全相同停止划分
    print(classList[2])
    print(classList[2])
    if classList.count(classList[0]) == len(classList):
        return classList[0]
    if len(dataSet[0]) == 1:
        return majorityCnt(classList)
    bestFeat = chooseBestFeatureToSplit(dataSet)
    bestFeatLabel = labels[bestFeat]
    myTree = {bestFeatLabel: {}}
    del(labels[bestFeat])
    featValues = [example[bestFeat] for example in dataSet]
    uniqueVals = set(featValues)
    for value in uniqueVals:
        sublabels = labels[:] 
        myTree[bestFeatLabel][value] = creatTree(
            splitDataSet(dataSet, bestFeat, value), sublabels)
    return myTree




In [22]:
tre  = creatTree(myData,labels)


['no', 'yes', 'yes', 'no', 'no', 'no', 'no', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'no']
yes
labelCounts {'no': 6, 'yes': 9}
labelCounts {'no': 3, 'yes': 2}
labelCounts {'no': 2, 'yes': 3}
labelCounts {'yes': 4, 'no': 1}
labelCounts {'no': 5, 'yes': 5}
labelCounts {'yes': 4, 'no': 1}
labelCounts {'no': 5, 'yes': 4}
labelCounts {'no': 1, 'yes': 5}
labelCounts {'no': 5}
labelCounts {'yes': 5, 'no': 1}
labelCounts {'yes': 4}
['no', 'no', 'no', 'no', 'no']
no
['yes', 'yes', 'no', 'yes', 'yes', 'yes']
no
labelCounts {'yes': 5, 'no': 1}
labelCounts {'yes': 2}
labelCounts {'no': 1, 'yes': 1}
labelCounts {'yes': 2}
labelCounts {'yes': 2, 'no': 1}
labelCounts {'yes': 3}
labelCounts {'yes': 3, 'no': 1}
labelCounts {'yes': 2}
['yes', 'yes']


IndexError: list index out of range

In [ ]:
# treePlotter.py
def plotMidText(cntrPt, parentPt, txtString):
    xMid = (parentPt[0]-cntrPt[0])/2.0 + cntrPt[0]
    yMid = (parentPt[1]-cntrPt[1])/2.0 + cntrPt[1]
    createPlot.ax1.text(xMid, yMid, txtString)


def plotTree(myTree, parentPt, nodeTxt):
    numLeafs = getTreeDepth(myTree)
    firstStr =  eval(str(myTree.keys()).replace('dict_keys(','').replace(')',''))[0]
    cntrPt = (plotTree.xOff+(1.0 + float(numLeafs))/
              2.0/plotTree.totalW, plotTree.yOff)
    plotMidText(cntrPt, parentPt, nodeTxt)
    plotNode(firstStr, cntrPt, parentPt, decisionNode)
    secondDict = myTree[firstStr]
    plotTree.yOff = plotTree.yOff - 1.0/plotTree.totalD
    for key in secondDict.keys():
        if type(secondDict[key]).__name__ == 'dict':
            plotTree(secondDict[key], cntrPt, str(key))
        else:
            plotTree.xOff = plotTree.xOff +1.0/plotTree.totalW
            plotNode(secondDict[key],(plotTree.xOff,plotTree.yOff),cntrPt,leafNode)
            plotMidText((plotTree.xOff,plotTree.yOff),cntrPt,str(key))
    plotTree.yOff = plotTree.yOff +1.0/plotTree.totalD

def createPlot(inTree):
    fig = plt.figure(1,facecolor='white')
    fig.clf()
    axprops = dict(xticks = [],yticks = [])
    createPlot.ax1 = plt.subplot(111,frameon = False,**axprops)
    plotTree.totalW = float(getNumLeaves(inTree))
    plotTree.totalD = float(getTreeDepth(inTree))
    plotTree.xOff = -0.5/plotTree.totalW
    plotTree.yOff = 1.0
    plotTree(inTree,(0.5,1.0),"")
    plt.show


createPlot(tre)








NameError: name 'plt' is not defined